# meta

> Metadata introspection for the NVIDIA monitor plugin used by cjm-ctl to generate the registration manifest

In [ ]:
#| default_exp meta

In [ ]:
#| export
import os
import sys
from typing import Dict, Any
from cjm_system_monitor_nvidia import __version__

## get_plugin_metadata

Returns metadata required to register this plugin with the PluginManager. This function is called by `cjm-ctl` during the manifest generation phase.

The returned dictionary includes:

- **python_path**: Absolute path to the environment's Python interpreter
- **module/class**: Import path for dynamic loading by the Universal Worker
- **resources**: Hardware requirements (this plugin doesn't require GPU itself)

In [ ]:
#| export
def get_plugin_metadata() -> Dict[str, Any]:  # Plugin metadata for manifest generation
    """Return metadata required to register this plugin with the PluginManager."""
    # Fallback base path (current behavior for backward compatibility)
    base_path = os.path.dirname(os.path.dirname(sys.executable))
    
    # Use CJM config if available, else fallback to env-relative paths
    cjm_data_dir = os.environ.get("CJM_DATA_DIR")
    
    # Plugin data directory
    plugin_name = "cjm-system-monitor-nvidia"
    if cjm_data_dir:
        data_dir = os.path.join(cjm_data_dir, plugin_name)
    else:
        data_dir = os.path.join(base_path, "data")
    
    db_path = os.path.join(data_dir, "nvidia_sysmon.db")
    
    # Ensure data directory exists
    os.makedirs(data_dir, exist_ok=True)
    
    return {
        "name": plugin_name,
        "version": __version__,
        "type": "infrastructure",
        "category": "system_monitor",
        "interface": "cjm_infra_plugin_system.plugin_interface.MonitorPlugin",
        
        "module": "cjm_system_monitor_nvidia.plugin",
        "class": "NvidiaMonitorPlugin",
        
        # Absolute path to this environment's Python interpreter
        "python_path": sys.executable,

        "db_path": db_path,
        
        # This plugin monitors GPUs but doesn't require GPU resources itself
        "resources": {
            "requires_gpu": False,
            "min_gpu_vram_mb": 0,
            "min_system_ram_mb": 128
        },
        
        "env_vars": {}
    }

## Testing

In [ ]:
import json

metadata = get_plugin_metadata()
print(json.dumps(metadata, indent=2))

{
  "name": "cjm-system-monitor-nvidia",
  "version": "0.0.3",
  "type": "infrastructure",
  "category": "system_monitor",
  "interface": "cjm_infra_plugin_system.plugin_interface.MonitorPlugin",
  "module": "cjm_system_monitor_nvidia.plugin",
  "class": "NvidiaMonitorPlugin",
  "python_path": "/home/innom-dt/miniforge3/envs/cjm-system-monitor-nvidia/bin/python3.11",
  "db_path": "/home/innom-dt/miniforge3/envs/cjm-system-monitor-nvidia/data/nvidia_sysmon.db",
  "resources": {
    "requires_gpu": false,
    "min_gpu_vram_mb": 0,
    "min_system_ram_mb": 128
  },
  "env_vars": {}
}


### CLI Introspection

The `cjm-ctl install-all` command runs this function inside the plugin's conda environment:

```bash
conda run -n cjm-sys-mon-nvidia python -c \
  'from cjm_system_monitor_nvidia.meta import get_plugin_metadata; \
   import json; print(json.dumps(get_plugin_metadata(), indent=2))'
```

The output is written to `~/.cjm/plugins/cjm-system-monitor-nvidia.json`.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()